# Airline delays 
## Bureau of Transportation Statistics
https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

~140GB

In [2]:
# imports
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [3]:
#dbutils.fs.ls('/databricks-datasets/airlines')

In [4]:
sum = 0
DATA_PATH = 'dbfs:/databricks-datasets/airlines/'
for item in dbutils.fs.ls(DATA_PATH):
  sum = sum+item.size
sum

Out[27]: 128911081815

In [5]:
# with open("/dbfs/databricks-datasets/airlines/README.md") as f:
#     x = ''.join(f.readlines())

# print(x)

In [6]:
from pyspark.sql.types import *
airline_schema = StructType([
            StructField("Year", IntegerType(), True),
            StructField("Month", IntegerType(), True),
            StructField("DayofMonth", IntegerType(), True),
            StructField("DayOfWeek", IntegerType(), True),
            StructField("DepTime", IntegerType(), True),
            StructField("CRSDepTime", IntegerType(), True),
            StructField("ArrTime", IntegerType(), True),
            StructField("CRSArrTime", IntegerType(), True),
            StructField("UniqueCarrier", StringType(), True),
            StructField("FlightNum", StringType(), True),
            StructField("TailNum", StringType(), True),
            StructField("ActualElapsedTime", IntegerType(), True),
            StructField("CRSElapsedTime", IntegerType(), True),
            StructField("AirTime", IntegerType(), True),
            StructField("ArrDelay", IntegerType(), True),
            StructField("DepDelay", IntegerType(), True),
            StructField("Origin", StringType(), True),
            StructField("Dest", StringType(), True),
            StructField("Distance", IntegerType(), True),
            StructField("TaxiIn", IntegerType(), True),			
			StructField("TaxiOut", IntegerType(), True),
            StructField("Cancelled", IntegerType(), True),
            StructField("CancellationCode", StringType(), True),
            StructField("Diverted", IntegerType(), True),
            StructField("CarrierDelay", IntegerType(), True),
            StructField("WeatherDelay", IntegerType(), True),
            StructField("NASDelay", IntegerType(), True),
            StructField("SecurityDelay", IntegerType(), True),
			StructField("LateAircraftDelay", IntegerType(), True),
            StructField("IsArrDelayed", StringType(), True),			
            StructField("IsDepDelayed", StringType(), True)])

In [7]:
airlines = spark.read.format("csv").option("header", "false").schema(airline_schema).load("dbfs:/databricks-datasets/airlines/part-*")
airlines.printSchema()

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: integer (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: integer (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: string (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: integer (nullable = true)
-- CRSElapsedTime: integer (nullable = true)
-- AirTime: integer (nullable = true)
-- ArrDelay: integer (nullable = true)
-- DepDelay: integer (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- TaxiIn: integer (nullable = true)
-- TaxiOut: integer (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: integer (nullable = true)
-- WeatherDelay: integer (nullable = true)
-- NASDelay: integer (nullable = true)
-- SecurityDelay: integer (nullable = true)
-- LateAircraftDelay: integer (nullable = true)
-- IsArrDelayed: string (nullable = true)
-- IsDepDelayed: string (nullable = true)

In [8]:
start_time = time.time()
count =  airlines.count()
total_time_csv = time.time() - start_time
print("Total records: {} \n Time taken (in seconds) to count total records: {}".format(count, np.round(total_time_csv, 2) )) 

Total records: 1235349691 
 Time taken (in seconds) to count total records: 55.73

In [9]:
airlines.write.parquet("dbfs:/tmp/parquet/airlines_data_in_parquet.parquet")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o8054.parquet.
: org.apache.spark.sql.AnalysisException: path dbfs:/tmp/parquet/airlines_data_in_parquet.parquet already exists.;
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:148)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:126)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:146)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:187)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:183)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:115)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:115)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:235)
	at org.apache.spark.sql.DataFrameWriter.parquet(DataFrameWriter.scala:600)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-2392600259027127> in <module> 
 ----> 1 airlines . write . parquet ( "dbfs:/tmp/parquet/airlines_data_in_parquet.parquet" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, path, mode, partitionBy, compression) 
 842 self . partitionBy ( partitionBy ) 
 843 self . _set_opts ( compression = compression ) 
 --> 844 self . _jwr

In [10]:
airlines_df = spark.read.parquet("dbfs:/tmp/parquet/airlines_data_in_parquet.parquet")
display(airlines_df)

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
2006,2,12,7,625,622,744,746,OO,6612,N746SK,79,84,55,-2,3,SLC,DEN,391,8,16,0,null,0,0,0,0,0,0,NO,YES
2006,2,12,7,1506,1457,1736,1737,OO,6614,N771SK,90,100,75,-1,9,DEN,MCI,533,4,11,0,null,0,0,0,0,0,0,NO,YES
2006,2,12,7,1148,1151,1330,1347,OO,6615,N910SW,102,116,89,-17,-3,DEN,GTF,624,4,9,0,null,0,0,0,0,0,0,NO,NO
2006,2,12,7,1445,1454,1623,1645,OO,6615,N910SW,98,111,84,-22,-9,GTF,DEN,624,7,7,0,null,0,0,0,0,0,0,NO,NO
2006,2,12,7,1013,1003,1258,1255,OO,6616,N710SK,105,112,90,3,10,DEN,CID,692,4,11,0,null,0,0,0,0,0,0,YES,YES
2006,2,12,7,1017,1021,1251,1250,OO,6618,N969SW,94,89,77,1,-4,DEN,FSD,483,4,13,0,null,0,0,0,0,0,0,YES,NO
2006,2,12,7,847,845,947,954,OO,6619,N958SW,120,129,103,-7,2,DEN,PSP,776,4,13,0,null,0,0,0,0,0,0,NO,YES
2006,2,12,7,1020,1024,1320,1325,OO,6619,N958SW,120,121,104,-5,-4,PSP,DEN,776,7,9,0,null,0,0,0,0,0,0,NO,NO
2006,2,12,7,907,910,1131,1142,OO,6620,N752SK,84,92,65,-11,-3,DEN,TUL,541,5,14,0,null,0,0,0,0,0,0,NO,NO
2006,2,12,7,1737,1735,1841,1843,OO,6621,N978SW,124,128,108,-2,2,DEN,PSP,776,4,12,0,null,0,0,0,0,0,0,NO,YES


In [11]:
airlines_df.printSchema()

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: integer (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: integer (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: string (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: integer (nullable = true)
-- CRSElapsedTime: integer (nullable = true)
-- AirTime: integer (nullable = true)
-- ArrDelay: integer (nullable = true)
-- DepDelay: integer (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- TaxiIn: integer (nullable = true)
-- TaxiOut: integer (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: integer (nullable = true)
-- WeatherDelay: integer (nullable = true)
-- NASDelay: integer (nullable = true)
-- SecurityDelay: integer (nullable = true)
-- LateAircraftDelay: integer (nullable = true)
-- IsArrDelayed: string (nullable = true)
-- IsDepDelayed: string (nullable = true)

In [12]:
start_time_parquet = time.time()
count = airlines_df.count()
total_time_parquet = time.time() - start_time_parquet
print("Total records: {} \n Time taken (in seconds) to count total records: {} \n Parquet read DataFrame is {} times faster than CSV read DataFrame".format(count, np.round(total_time_parquet, 2), np.round(total_time_csv/total_time_parquet, 2) )) 

Total records: 1235349691 
 Time taken (in seconds) to count total records: 0.69 
 Parquet read DataFrame is 80.64 times faster than CSV read DataFrame

In [13]:
null_feature_list = []
for column in airlines_df.columns:
    nulls = airlines_df.filter(airlines_df[column].isNull()).count()
    nulls_perct = np.round((nulls/count)*100, 2)
    null_feature_list.append([column, nulls, nulls_perct])
  

In [14]:
nullCounts_df = pd.DataFrame(np.array(null_feature_list), columns=['Feature_Name', 'Null_Counts', 'Percentage_Null_Counts'])
nullCounts_df

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,Year,899947541,72.85
1,Month,899947541,72.85
2,DayofMonth,899947541,72.85
3,DayOfWeek,899947541,72.85
4,DepTime,899947541,72.85
5,CRSDepTime,899947541,72.85
6,ArrTime,899947541,72.85
7,CRSArrTime,899947541,72.85
8,UniqueCarrier,899947541,72.85
9,FlightNum,899947541,72.85


In [15]:
Delay_List = ['ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay' ]
airlines_df[Delay_List].describe().show()

+-------+------------------+------------------+-----------------+------------------+------------------+--------------------+
summary| ArrDelay| DepDelay| CarrierDelay| WeatherDelay| NASDelay| SecurityDelay|
+-------+------------------+------------------+-----------------+------------------+------------------+--------------------+
 count| 335402150| 335402150| 335402150| 335402150| 335402150| 335402150|
 mean| 9.96252125992633| 10.85921363354409|3.755347960649626|0.8039785672214683| 4.184948426836262|0.027236557666669697|
 stddev|38.100658820119044|34.838956662035535|20.04748170258808| 9.526800022245132|16.748927012787277| 1.2055792150980416|
 min| -1302| -1200| 0| 0| -60| 0|
 max| 2598| 2601| 2580| 1510| 1392| 533|
+-------+------------------+------------------+-----------------+------------------+------------------+--------------------+

In [16]:
sample_airlines_df = airlines_df.filter(airlines_df.ArrDelay != 'NaN').select(["ArrDelay"]).sample(False, 0.001, 42)
pandas_df = sample_airlines_df.toPandas()

In [17]:
sns.distplot(pandas_df)

plt.show()

--------------------------------------------------------------------------- 
 ZeroDivisionError Traceback (most recent call last)
 <command-2392600259027417> in <module> 
 ----> 1 sns . distplot ( pandas_df ) 
 2 
 3 plt . show ( ) 

 /databricks/python/lib/python3.7/site-packages/seaborn/distributions.py in distplot (a, bins, hist, kde, rug, fit, hist_kws, kde_kws, rug_kws, fit_kws, color, vertical, norm_hist, axlabel, label, ax) 
 196 line , = ax . plot ( 0 , a . mean ( ) ) 
 197 else : 
 --> 198 line , = ax . plot ( a . mean ( ) , 0 ) 
 199 color = line . get_color ( ) 
 200 line . remove ( ) 

 /databricks/python/lib/python3.7/site-packages/numpy/core/_methods.py in _mean (a, axis, dtype, out, keepdims) 
 85 ret = ret . dtype . type ( ret / rcount ) 
 86 else : 
 ---> 87 ret = ret / rcount
 88 
 89 return ret

 ZeroDivisionError : division by zero